In [1]:
#Please select GPU first (from Edit->NotebookSetting)
import pandas as pd
import numpy as np
import io
import os
import re
from google.colab import drive
drive.mount('/content/drive')
!pip install fastai==2.3.1
from fastai.text.all import *

Mounted at /content/drive
     |████████████████████████████████| 204kB 7.6MB/s 
     |████████████████████████████████| 61kB 7.8MB/s 
     |████████████████████████████████| 804.1MB 23kB/s 
ERROR: torchvision 0.10.0+cu102 has requirement torch==1.9.0, but you'll have torch 1.8.1 which is incompatible.
ERROR: torchtext 0.10.0 has requirement torch==1.9.0, but you'll have torch 1.8.1 which is incompatible.
  Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


In [2]:
path = '/content/drive/MyDrive/colab_data'
def de_emojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')
def clean_text(text):
    text = re.sub(r"@[A-Za-z0-9]+", ' ', text)
    text = re.sub(r"https?://[A-Za-z0-9./]+", ' ', text)
    text = re.sub(r"[^a-zA-z.!?'0-9]", ' ', text)
    text = re.sub('\t', ' ',  text)
    text = re.sub(r" +", ' ', text)
    return text
def text_proc(df, text_col='text'):
    df['orig_text'] = df[text_col]
    # Remove twitter handles
    df[text_col] = df[text_col].apply(lambda x: clean_text(x))
    # Remove URLs
    df[text_col] = df[text_col].apply(lambda x:x.replace('<br />', ' '))
    return df[df[text_col]!='']
data = pd.read_csv(os.path.join(path, "nasdaq.csv"))
data = text_proc(data,'Headline').dropna(subset=['Label'])#.sample(2000, random_state = 10).reset_index(drop=True)
data['len'] = data.Headline.apply(lambda x: len(x.split(' ')))
print(len(data))
print(data['len'].quantile(0.99))
data.head(3)

13181
52.0


,Label,Ticker,Headline,orig_text,len
0,0,A,Airline shares tumble as New York imposes quarantine Florida cases spike An American Airlines Group Inc. plane prepares for landing at LaGuardia Airport LGA in New York U.S. on Tuesday April 18... TravelNewsInsights,@TotesTravel : Airline shares tumble as New York imposes quarantine Florida cases spike - An American Airlines Group Inc. plane prepares for landing at LaGuardia Airport (LGA) in New York U.S. on Tuesday April 18... #TravelNewsInsights - https://t.co/6fnJSf3bRh,35
1,1,A,American United call off Hong Kong flights after crew testing rules An American Airlines Group Inc. Boeing Co. 777 300ER aircraft sits at Hong Kong International Airport in Hong Kong China. Brent... TravelNewsInsights,@TotesTravel : American United call off Hong Kong flights after crew testing rules - An American Airlines Group Inc. Boeing Co. 777-300ER aircraft sits at Hong Kong International Airport in Hong Kong China. Brent... #TravelNewsInsights - https://t.co/mM34IZz6lM,35
2,0,A,U.S. airline stocks hit highest prices since June on travel uptick stimulus hopes A JetBlue Airways Corp. plane taxis next to American Airlines Group Inc. Delta Air Lines Inc. and Alaska Airlines Inc. aircraft... TravelNewsInsights,@TotesTravel : U.S. airline stocks hit highest prices since June on travel uptick stimulus hopes - A JetBlue Airways Corp. plane taxis next to American Airlines Group Inc. Delta Air Lines Inc. and Alaska Airlines Inc. aircraft... #TravelNewsInsights - https://t.co/bwxoJDizzN,37


In [12]:
df_clas = data[['Headline', 'Label']].dropna(subset=['Label'])
df_clas.columns = df_clas.columns.str.strip()

In [3]:
dls_lm = TextDataLoaders.from_df(data, text_col='Headline', is_lm=True, valid_pct=0.1)
dls_lm.show_batch(max_n=2)
torch.save(dls_lm, os.path.join(path, 'dls_lm.pkl'))

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


,text,text_
0,xxbos xxmaj netgear xxmaj router xxmaj help amp amp xxmaj support in xxup usa 1 xxunk xxunk xxunk xxbos xxmaj boris xxmaj johnson is a known liar who is copying xxmaj vladimir xxmaj putin s approach to the media according to xxmaj channel 4 s head of news and current affairs who called on broadcasters to aggressively challenge the prime minister s falsehoods . xxbos xxup n. xxmaj korea cancels xxmaj wednesday,xxmaj netgear xxmaj router xxmaj help amp amp xxmaj support in xxup usa 1 xxunk xxunk xxunk xxbos xxmaj boris xxmaj johnson is a known liar who is copying xxmaj vladimir xxmaj putin s approach to the media according to xxmaj channel 4 s head of news and current affairs who called on broadcasters to aggressively challenge the prime minister s falsehoods . xxbos xxup n. xxmaj korea cancels xxmaj wednesday 's
1,xxmaj yr xxmaj old xxmaj egyptian xxmaj seal in xxmaj israel xxmaj xxunk in the shape of a beetle xxmaj the xxunk represents xxmaj xxunk xxmaj xxunk xxrep 3 i xxunk xxunk xxup bc sitting on his throne . xxmaj he set up govt in xxmaj canaan amp xxunk wars including battle of xxmaj xxunk a victory xxunk on xxmaj xxunk xxmaj temple xxunk . xxbos xxmaj pair guilty of xxunk xxmaj,yr xxmaj old xxmaj egyptian xxmaj seal in xxmaj israel xxmaj xxunk in the shape of a beetle xxmaj the xxunk represents xxmaj xxunk xxmaj xxunk xxrep 3 i xxunk xxunk xxup bc sitting on his throne . xxmaj he set up govt in xxmaj canaan amp xxunk wars including battle of xxmaj xxunk a victory xxunk on xxmaj xxunk xxmaj temple xxunk . xxbos xxmaj pair guilty of xxunk xxmaj dave


In [4]:
learn = language_model_learner(dls_lm, AWD_LSTM, drop_mult = 0.3, metrics=[accuracy, Perplexity()]).to_fp16()

In [5]:
learn.unfreeze()
learn.fit_one_cycle(4, 1e-3) 

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.589721,4.210132,0.328730,67.365410,00:19
1,4.034225,3.759761,0.372730,42.938156,00:19
2,3.521107,3.526779,0.406894,34.014244,00:19
3,3.183434,3.500624,0.411770,33.136135,00:19


In [6]:
TEXT = "I love"
N_WORDS = 45
N_SENTENCES = 2
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

i love things like China and Japan in the world of democracy Sary Ptc Group Inc to Prevent Its Employees from Communicating with Their Own Phone Systems Inc .
i love S. Korea only after giving birth to baby girl The World Bank says it will shrink its financial institutions over the next two months and that the number of foreign financial institutions in the region will be up to


In [15]:
dls_clas = DataBlock(
    blocks = (TextBlock.from_df('Headline', seq_len = dls_lm.seq_len, vocab = dls_lm.vocab), CategoryBlock),
    get_x = ColReader('text'),
    get_y = ColReader('Label'),
    splitter = RandomSplitter()
).dataloaders(df_clas, bs = 64)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [16]:
dls_clas.show_batch(max_n=4)

,text,category
0,xxbos xxup free xxup xxunk xxup riot xxmaj campaign gets a big dose of xxunk power with the help of xxmaj xxunk xxmaj xxunk xxmaj jones xxmaj kate xxmaj nash xxmaj the xxmaj knife xxmaj xxunk xxmaj li xxmaj xxunk xxmaj snow xxmaj peter xxmaj xxunk and xxmaj john the xxmaj yeah xxmaj yeah xxmaj xxunk xxmaj margaret xxmaj cho xxmaj dave xxmaj hill xxmaj motor xxmaj city 5 xxmaj the xxmaj xxunk xxmaj xxunk of xxmaj death xxunk xxmaj sisters xxup you and many more !,0
1,xxbos xxup hardware xxup features xxmaj xxunk l x w x h 440 mm x xxunk mm x 44 mm l w h xxmaj interface xxmaj fixed xxmaj ports 16 10 xxunk xxup tx ports 2 10 100 1 xxrep 3 0 xxmaj base t ports 2 1 xxrep 3 0 xxmaj base x xxup xxunk ports xxmaj combo lightning protection xxunk xxup led xxmaj link xxmaj act xxunk xxmaj status xxmaj system xxmaj power xxunk xxmaj max xxmaj power xxmaj supply xxmaj power supply xxunk,0
2,xxbos xxmaj extinct xxmaj cave xxmaj bear xxup dna xxmaj found in xxmaj living xxmaj bears xxmaj discovery is 1st of its kind outside human lineage . xxmaj after roaming for over 1 xxrep 5 0 yrs cave bears died out some 24 xxrep 3 0 yrs ago . xxmaj but its xxup dna lives on in brown bears . xxmaj it forces us to think on a philosophical level what we mean by species extinction .,0
3,xxbos xxmaj legal xxmaj case xxmaj filed xxmaj against 4 xxup u.s . xxmaj presidents and 4 xxup uk xxmaj prime xxmaj ministers xxmaj in xxmaj spain xxmaj this case naming xxmaj george h w xxmaj bush xxmaj william j xxmaj clinton xxmaj george w xxmaj bush xxmaj barack h xxmaj obama xxmaj margaret xxmaj thatcher xxmaj john xxmaj major xxmaj anthony xxmaj blair and xxmaj gordon xxmaj brown is brought by xxmaj iraqis and others,1


In [17]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, metrics=accuracy).to_fp16()

In [18]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.051414,0.930666,0.568665,00:17
1,0.865199,0.835068,0.627845,00:17
